1. Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных. 
2. Подобрать необходимое распределение и тип связи, при необходимости ознакомиться с документацией H20. 
3. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.). 
4. Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке).
5. Проанализировать вероятные проблемы. 
6. Предложить способы их решения и/или попробовать их решить, улучшив результат.

In [1]:
# !pip install h2o
import numpy as np
import pandas as pd
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [38]:
df2 = pd.read_csv('~/Yandex.Disk/geekbrains/Python/MLInBusines/lesson_6/freMPL-R.csv', low_memory=False)

In [50]:
for rec in df2.to_dict(orient='records'):
    print(rec)
    break

{'Exposure': 0.583, 'LicAge': 366, 'RecordBeg': '2004-06-01', 'RecordEnd': nan, 'VehAge': '2', 'Gender': 'Female', 'MariStat': 'Other', 'SocioCateg': 'CSP1', 'VehUsage': 'Professional', 'DrivAge': 55, 'HasKmLimit': 0, 'BonusMalus': 72, 'VehBody': 'sedan', 'VehPrice': 'D ', 'VehEngine': 'injection', 'VehEnergy': 'regular', 'VehMaxSpeed': '160-170 km/h', 'VehClass': 'B', 'ClaimAmount': 0.0, 'RiskVar': 15.0, 'Garage': 'None', 'ClaimInd': 0, 'Dataset': 1, 'DeducType': nan, 'ClaimNbResp': nan, 'ClaimNbNonResp': nan, 'ClaimNbParking': nan, 'ClaimNbFireTheft': nan, 'ClaimNbWindscreen': nan, 'OutUseNb': nan, 'RiskArea': nan}


In [40]:
df2.to_dict

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343080 entries, 0 to 343079
Data columns (total 31 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           343080 non-null  float64
 1   LicAge             343080 non-null  int64  
 2   RecordBeg          343080 non-null  object 
 3   RecordEnd          181115 non-null  object 
 4   VehAge             177880 non-null  object 
 5   Gender             343080 non-null  object 
 6   MariStat           343080 non-null  object 
 7   SocioCateg         343080 non-null  object 
 8   VehUsage           343080 non-null  object 
 9   DrivAge            343080 non-null  int64  
 10  HasKmLimit         343080 non-null  int64  
 11  BonusMalus         343080 non-null  int64  
 12  VehBody            145780 non-null  object 
 13  VehPrice           145780 non-null  object 
 14  VehEngine          145780 non-null  object 
 15  VehEnergy          145780 non-null  object 
 16  Ve

In [2]:
df = pd.read_csv('~/Yandex.Disk/geekbrains/Python/MLInBusines/lesson_6/freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Exposure           115155 non-null  float64
 1   LicAge             115155 non-null  int64  
 2   RecordBeg          115155 non-null  object 
 3   RecordEnd          59455 non-null   object 
 4   Gender             115155 non-null  object 
 5   MariStat           115155 non-null  object 
 6   SocioCateg         115155 non-null  object 
 7   VehUsage           115155 non-null  object 
 8   DrivAge            115155 non-null  int64  
 9   HasKmLimit         115155 non-null  int64  
 10  BonusMalus         115155 non-null  int64  
 11  ClaimAmount        115155 non-null  float64
 12  ClaimInd           115155 non-null  int64  
 13  ClaimNbResp        115155 non-null  float64
 14  ClaimNbNonResp     115155 non-null  float64
 15  ClaimNbParking     115155 non-null  float64
 16  Cl

In [3]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0]


,ClaimAmount,ClaimInd
82,-74.206042,0
175,-1222.585196,0
177,-316.288822,0
363,-666.758610,0
375,-1201.600604,0


In [4]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

In [5]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [6]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Male', 1: 'Female'}
{0: 'Other', 1: 'Alone'}


In [7]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [8]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


In [9]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [10]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,14
CSP3,1210
CSP1,2740
CSP2,3254
CSP4,7648
CSP6,24833
CSP5,75456


In [11]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

In [12]:
df = df.select_dtypes(exclude=['object'])

In [13]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,...,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
1,0.916,333,0,0,46,0,50,0.0,0,0.0,...,1,0,0,0,0,0,1,0,0,2116
2,0.550,173,0,0,32,0,68,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,1024
3,0.089,364,1,0,52,0,50,0.0,0,0.0,...,0,0,0,0,0,0,1,0,0,2704
4,0.233,426,0,0,57,0,50,0.0,0,0.0,...,0,0,0,0,0,0,0,1,0,3249


In [15]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [16]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,104286
2.0,3529
1.0,3339
3.0,2310
4.0,1101
5.0,428
6.0,127
7.0,26
8.0,6
9.0,2


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115155 entries, 0 to 115154
Data columns (total 24 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   Exposure                         115155 non-null  float64
 1   LicAge                           115155 non-null  int64  
 2   Gender                           115155 non-null  int64  
 3   MariStat                         115155 non-null  int64  
 4   DrivAge                          115155 non-null  int64  
 5   HasKmLimit                       115155 non-null  int64  
 6   BonusMalus                       115155 non-null  int64  
 7   ClaimAmount                      115155 non-null  float64
 8   ClaimInd                         115155 non-null  int64  
 9   OutUseNb                         115155 non-null  float64
 10  RiskArea                         115155 non-null  float64
 11  VehUsage_Private                 115155 non-null  uint8  
 12  Ve

In [18]:
# Разбиение датасета на train/val/test
# , 'CountPredicted', 'AvgClaimPredicted', 'BurningCost'
x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1)

In [19]:
x_train_ind.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80608 entries, 54963 to 98539
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Exposure                         80608 non-null  float64
 1   LicAge                           80608 non-null  int64  
 2   Gender                           80608 non-null  int64  
 3   MariStat                         80608 non-null  int64  
 4   DrivAge                          80608 non-null  int64  
 5   HasKmLimit                       80608 non-null  int64  
 6   BonusMalus                       80608 non-null  int64  
 7   OutUseNb                         80608 non-null  float64
 8   RiskArea                         80608 non-null  float64
 9   VehUsage_Private                 80608 non-null  uint8  
 10  VehUsage_Private+trip to office  80608 non-null  uint8  
 11  VehUsage_Professional            80608 non-null  uint8  
 12  VehUsage_Profe

In [20]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu219.10); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu219.10, mixed mode, sharing)
  Starting server from /home/aleksandr/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmptsdszxys
  JVM stdout: /tmp/tmptsdszxys/h2o_aleksandr_started_from_python.out
  JVM stderr: /tmp/tmptsdszxys/h2o_aleksandr_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Asia/Aqtobe
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,22 days
H2O_cluster_name:,H2O_from_python_aleksandr_eho0fx
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.902 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [22]:
# Преобразование в H2O-Frame
h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [42]:
x_test_ind.to_dict(orient='records')

[{'Exposure': 0.083,
  'LicAge': 242,
  'Gender': 1,
  'MariStat': 1,
  'DrivAge': 38,
  'HasKmLimit': 0,
  'BonusMalus': 57,
  'OutUseNb': 0.0,
  'RiskArea': 8.0,
  'VehUsage_Private': 0,
  'VehUsage_Private+trip to office': 1,
  'VehUsage_Professional': 0,
  'VehUsage_Professional run': 0,
  'SocioCateg_CSP1': 0,
  'SocioCateg_CSP2': 0,
  'SocioCateg_CSP3': 0,
  'SocioCateg_CSP4': 0,
  'SocioCateg_CSP5': 1,
  'SocioCateg_CSP6': 0,
  'SocioCateg_CSP7': 0,
  'DrivAgeSq': 1444},
 {'Exposure': 0.638,
  'LicAge': 67,
  'Gender': 0,
  'MariStat': 1,
  'DrivAge': 24,
  'HasKmLimit': 0,
  'BonusMalus': 90,
  'OutUseNb': 0.0,
  'RiskArea': 11.0,
  'VehUsage_Private': 0,
  'VehUsage_Private+trip to office': 1,
  'VehUsage_Professional': 0,
  'VehUsage_Professional run': 0,
  'SocioCateg_CSP1': 0,
  'SocioCateg_CSP2': 0,
  'SocioCateg_CSP3': 0,
  'SocioCateg_CSP4': 0,
  'SocioCateg_CSP5': 1,
  'SocioCateg_CSP6': 0,
  'SocioCateg_CSP7': 0,
  'DrivAgeSq': 576},
 {'Exposure': 0.5,
  'LicAge': 375,

In [37]:
x_test_ind.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17274 entries, 19322 to 50934
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Exposure                         17274 non-null  float64
 1   LicAge                           17274 non-null  int64  
 2   Gender                           17274 non-null  int64  
 3   MariStat                         17274 non-null  int64  
 4   DrivAge                          17274 non-null  int64  
 5   HasKmLimit                       17274 non-null  int64  
 6   BonusMalus                       17274 non-null  int64  
 7   OutUseNb                         17274 non-null  float64
 8   RiskArea                         17274 non-null  float64
 9   VehUsage_Private                 17274 non-null  uint8  
 10  VehUsage_Private+trip to office  17274 non-null  uint8  
 11  VehUsage_Professional            17274 non-null  uint8  
 12  VehUsage_Profe

In [23]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных
h2o_train_ind['ClaimInd'] = h2o_train_ind['ClaimInd'].asfactor()
h2o_valid_ind['ClaimInd'] = h2o_valid_ind['ClaimInd'].asfactor()
h2o_test_ind['ClaimInd'] = h2o_test_ind['ClaimInd'].asfactor()

In [24]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson = H2OGeneralizedLinearEstimator(family = "binomial", nfolds=5)

In [25]:
glm_poisson.train(y="ClaimInd", x = h2o_train_ind.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [26]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,"Elastic Net (alpha = 0.5, lambda = 3.295E-5 )",20,19,3,py_1_sid_a197


In [27]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson.cross_validation_metrics_summary()


Cross-Validation Metrics Summary: 


,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.4367453,0.081059344,0.46229348,0.35844737,0.48086718,0.5346413,0.34747702
1,auc,0.5672994,0.008221347,0.5736072,0.5703258,0.57399344,0.5543785,0.5641921
2,aucpr,0.15534346,0.0070099453,0.16512938,0.15654197,0.15785395,0.14759663,0.14959536
3,err,0.5632547,0.081059344,0.53770655,0.6415526,0.5191328,0.46535867,0.652523
4,err_count,4018.1353,592.4557,3817.44,4577.007,3733.977,3285.456,4676.796
5,f0point5,0.17358798,0.0053999424,0.1805168,0.17525868,0.17586613,0.169011,0.16728726
6,f1,0.24113674,0.008227779,0.24863276,0.24800166,0.24247424,0.22868001,0.23789501
7,f2,0.39573815,0.026827954,0.39930654,0.4239785,0.39029643,0.353473,0.41163632
8,lift_top_group,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,logloss,0.3800971,0.006228407,0.3875543,0.3857664,0.37588263,0.37341085,0.37787136



See the whole table with table.as_data_frame()


In [28]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_poisson._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.174604,-1.939603
1,LicAge,-0.000189,-0.030147
2,Gender,0.015184,0.007353
3,MariStat,-0.052343,-0.018717
4,DrivAge,-0.003914,-0.058577
5,HasKmLimit,-0.425867,-0.132326
6,BonusMalus,0.006630,0.099958
7,OutUseNb,0.068515,0.045966
8,RiskArea,0.014767,0.032670
9,VehUsage_Private,-0.157718,-0.074800


In [29]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson.coef_norm()
for x in range(len(glm_poisson.cross_validation_models())):
    pmodels[x] = glm_poisson.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-1.93960,-1.94798,-1.94626,-1.93505,-1.93257,-1.93752
LicAge,-0.03015,-0.02147,-0.01899,-0.03550,-0.02334,-0.04600
Gender,0.00735,0.00992,0.00684,0.00419,0.00928,0.00621
MariStat,-0.01872,-0.00454,-0.02473,-0.02122,-0.02606,-0.01748
DrivAge,-0.05858,-0.08781,-0.06547,-0.06215,-0.06867,-0.04094
HasKmLimit,-0.13233,-0.11438,-0.12803,-0.14504,-0.13491,-0.13947
BonusMalus,0.09996,0.09222,0.11312,0.09438,0.08914,0.10966
OutUseNb,0.04597,0.04902,0.03653,0.04568,0.04781,0.05050
RiskArea,0.03267,0.03008,0.04400,0.01980,0.04293,0.02648
VehUsage_Private,-0.07480,-0.05988,-0.07634,-0.06793,-0.06588,-0.07262


In [30]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

c_train_pred = glm_poisson.predict(h2o_train_ind).as_data_frame()
c_valid_pred = glm_poisson.predict(h2o_valid_ind).as_data_frame()
c_test_pred = glm_poisson.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [35]:
model_glm_poisson = h2o.save_model(model=glm_poisson, 
                                   path="/home/aleksandr/Yandex.Disk/github/MLInBusines/project/",
                                   force=True)

In [31]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок
true_test = h2o_test_ind['ClaimInd'].as_data_frame()['ClaimInd'].values
pred_test = c_test_pred['predict'].values
true_train = h2o_train_ind['ClaimInd'].as_data_frame()['ClaimInd'].values
pred_train = c_train_pred['predict'].values
# accuracy_score, f1_score, confusion_matrix
acu_train = accuracy_score(true_train, pred_train)
f1_train = f1_score(true_train, pred_train)
matr_train = confusion_matrix(true_train, pred_train)
acu_test = accuracy_score(true_test, pred_test)
f1_test = f1_score(true_test, pred_test)
matr_test = confusion_matrix(true_test, pred_test)

In [36]:
print(f'Accuracy score train: {acu_train}')
print(f'F_score train: {f1_train}')
print(f'confusion_matrix train: \n{matr_train}')
print(f'Accuracy score test: {acu_test}')
print(f'F_score test: {f1_test}')
print(f'confusion_matrix test: \n{matr_test}')

Accuracy score train: 0.5353066693132196
F_score train: 0.18459663024075926
confusion_matrix train: 
[[38910 34063]
 [ 3395  4240]]
Accuracy score test: 0.536934120643742
F_score test: 0.18568665377176016
confusion_matrix test: 
[[8363 7286]
 [ 713  912]]


In [37]:
# Сохранение обученной модели
model_glm_poisson = h2o.save_model(model=glm_poisson, path="model_lesson_7", force=True)

In [38]:
df['ClaimInd'].value_counts()

0    104286
1     10869
Name: ClaimInd, dtype: int64

Результаты работы модели неудовлетворительные, возможной причиной может быть не сбалансированность классов. Не успел изучить влияние балансировки классов на работу модели. Также для улучшения качества модели можно поэксперементировать с генерацией новых признаков.

In [32]:
pred_test

array([1, 1, 1, ..., 0, 0, 1])